In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Load the data
data_path = ''
data = pd.read_csv(data_path)


data.replace([np.inf, -np.inf], np.nan, inplace=True)

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Assume 'label' is the name of the column to predict
X = data.drop('label', axis=1)
y = data['label']



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Extra Trees Classifier
extra_trees_model = ExtraTreesClassifier(random_state=42)
extra_trees_model.fit(X_train, y_train)

# Make predictions
y_pred = extra_trees_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Get feature importances
importances = extra_trees_model.feature_importances_

# Create a DataFrame of features and their importances
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# Sort the DataFrame by importance in descending order
feature_importances_sorted = feature_importances.sort_values(by='importance', ascending=False)

# Print sorted feature importances
print("Sorted Feature Importances:")
print(feature_importances_sorted)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize results list to store accuracies for different feature counts
results = []

# Loop through feature subsets from top 5 to top 15 features
for top_n in range(15, 16):
    # Select the top 'top_n' features based on importance
    top_features = feature_importances_sorted['feature'].head(top_n)
    
    # Subset the training and testing sets to the top 'top_n' features
    X_train_reduced = X_train[top_features]
    X_test_reduced = X_test[top_features]

    # Scale the reduced feature sets
    scaler = StandardScaler()
    X_train_reduced_scaled = scaler.fit_transform(X_train_reduced)
    X_test_reduced_scaled = scaler.transform(X_test_reduced)

    # Reinitialize and retrain the Extra Trees Classifier on reduced feature set
    model_reduced = ExtraTreesClassifier(random_state=42)
    model_reduced.fit(X_train_reduced_scaled, y_train)
    
    # Make predictions with the reduced model
    y_pred_reduced = model_reduced.predict(X_test_reduced_scaled)
    
    # Calculate and store accuracy
    accuracy_reduced = accuracy_score(y_test, y_pred_reduced)
    results.append((top_n, accuracy_reduced))
    
    # Print the accuracy after each model run
    print(f"Top {top_n} Features Model Accuracy: {accuracy_reduced:.4f}")

# Print the final list of accuracies for each feature count
print("\nFinal List of Accuracies for Each Feature Count:")
for result in results:
    print(f"Top {result[0]} Features: Accuracy = {result[1]:.4f}")
